In [65]:
!pip install apriltag

In [55]:
!pip install gtsam

In [56]:
import cv2
import apriltag
import gtsam
import numpy as np

In [57]:
# camera matrix
k = np.array([[1476.36024, 0 , 979.133964], [0,1482.182676 , 552.332448],[0, 0, 1]])

In [58]:
# Create a GTSAM Cal3_S2 calibration object using the camera matrix parameters
calibration = gtsam.Cal3_S2(k[0,0],k[1,1],0.0,k[0,2],k[1,2])

In [59]:
# Load the image
image_path = "/content/drive/MyDrive/vslam/vslam/frame_0.jpg"
image = cv2.imread(image_path)
gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

In [60]:
# Create an AprilTag detector instance
detector = apriltag.Detector()

# Detect AprilTags in the grayscale image
tags = detector.detect(gray)

# Assume the first detected tag for further processing
tag = tags[0]

In [61]:
# Extract the corner coordinates of the detected AprilTag
corners = np.array(tag.corners)

# Extract the center coordinates of the detected AprilTag
center_x, center_y = tag.center

In [62]:
# Set the size of the AprilTag in meters
tag_size = 0.01

# Define the pose of the AprilTag's origin in the camera frame
tag_origin = gtsam.Pose3(gtsam.Rot3(), gtsam.Point3(0, 0, 0))


In [63]:
# Initialize the graph for factor graph optimization
factor_graph = gtsam.NonlinearFactorGraph()

In [64]:
# Initialize loop variable
corner_index = 0

# Assuming 'corners' is defined outside the loop
while corner_index < 4:
    # Extract measured corner
    measured_corner = corners[corner_index]

    # Define keys for point and pose in GTSAM
    point_key = gtsam.symbol('p', 3 - corner_index)
    pose_key = gtsam.symbol('x', 0)

    # Create a generic projection factor for camera calibration
    projection_factor = gtsam.GenericProjectionFactorCal3_S2(
        measured_corner,
        gtsam.noiseModel.Isotropic.Sigma(2, 0.00001),
        pose_key,
        point_key,
        calibration
    )

    # Add the projection factor to the graph
    factor_graph.add(projection_factor)

    # Compute width and depth for 3D point initialization
    width = np.sqrt((corners[0][0] - corners[3][0]) ** 2 + (corners[0][1] - corners[3][1]) ** 2)
    depth = (tag_size * 1473.33782) / width

    # Calibrate center coordinates
    calibrated_cent_x, calibrated_cent_y = calibration.calibrate(gtsam.Point2(center_x, center_y))

    # Define constrained noise model for 3D point priors
    constrained_noise = gtsam.noiseModel.Constrained.All(3)

    # Define 3D coordinates of tag corners
    tag_corners_3d = [[-tag_size / 2, tag_size / 2, 0],
                      [tag_size / 2, tag_size / 2, 0],
                      [tag_size / 2, -tag_size / 2, 0],
                      [-tag_size / 2, -tag_size / 2, 0]]

    # Loop through the corners for adding prior factors
    prior_index = 0
    while prior_index < 4:
        prior_factor = gtsam.PriorFactorPoint3(gtsam.symbol('p', prior_index), tag_corners_3d[prior_index], constrained_noise)
        factor_graph.add(prior_factor)
        prior_index += 1

    # Initialize values for optimization
    initial_estimate = gtsam.Values()

    # Initialize pose and 3D points
    initial_pose = gtsam.Pose3(gtsam.Rot3(), gtsam.Point3(-calibrated_cent_x, -calibrated_cent_y, -depth))
    initial_estimate.insert(gtsam.symbol('x', 0), initial_pose)

    prior_index = 0
    while prior_index < 4:
        initial_estimate.insert(gtsam.symbol('p', prior_index), gtsam.Point3(*tag_corners_3d[prior_index]))
        prior_index += 1

    # Create Levenberg-Marquardt optimizer
    optimizer = gtsam.LevenbergMarquardtOptimizer(factor_graph, initial_estimate)

    # Optimize the graph
    optimized_values = optimizer.optimize()

    # Increment loop variable
    corner_index += 1

# Extract optimized camera pose
optimized_camera_pose = optimized_values.atPose3(gtsam.symbol('x', 0))

# Print the optimized camera pose
print("Optimized Camera Pose:")
print(optimized_camera_pose)


Optimized Camera Pose:
R: [
	0.992071, -0.0384934, 0.119635;
	0.043614, 0.998228, -0.0404812;
	-0.117864, 0.045378, 0.991992
]
t: 0.000402179  0.00959959  -0.0833979



In [ ]:
import cv2
import apriltag
from google.colab.patches import cv2_imshow

# construct the argument parser and parse the arguments
image_path = "/content/drive/MyDrive/vslam/vslam/frame_0.jpg"

print("[INFO] loading image...")
image = cv2.imread(image_path)
gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

# define the AprilTags detector options and then detect the AprilTags
# in the input image
print("[INFO] detecting AprilTags...")
options = apriltag.DetectorOptions(families="tag36h11")
detector = apriltag.Detector(options)
results = detector.detect(gray)
print("[INFO] {} total AprilTags detected".format(len(results)))

# loop over the AprilTag detection results
for r in results:
  tag_id = r.tag_id
  if tag_id == 0:
    # extract the bounding box (x, y)-coordinates for the AprilTag
    # and convert each of the (x, y)-coordinate pairs to integers
    (ptA, ptB, ptC, ptD) = r.corners
    ptB = (int(ptB[0]), int(ptB[1]))
    ptC = (int(ptC[0]), int(ptC[1]))
    ptD = (int(ptD[0]), int(ptD[1]))
    ptA = (int(ptA[0]), int(ptA[1]))
    # draw the bounding box of the AprilTag detection
    cv2.line(image, ptA, ptB, (0, 255, 0), 2)
    cv2.line(image, ptB, ptC, (0, 255, 0), 2)
    cv2.line(image, ptC, ptD, (0, 255, 0), 2)
    cv2.line(image, ptD, ptA, (0, 255, 0), 2)
    # draw the center (x, y)-coordinates of the AprilTag
    (cX, cY) = (int(r.center[0]), int(r.center[1]))
    cv2.circle(image, (cX, cY), 5, (0, 0, 255), -1)
    # draw the tag family on the image
    tagFamily = r.tag_family.decode("utf-8")
    cv2.putText(image, tagFamily, (ptA[0], ptA[1] - 15),
      cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)
    print("[INFO] tag family: {}".format(tagFamily))
  # show the output image after AprilTag detection
  cv2_imshow(image)
  # cv2.waitKey(0)